In [1]:
import numpy as np
import pyedflib
import statistics
import plotly.graph_objects as go
import pandas as pd
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy, Amplitude, NumberOfPoints, ComplexPolynomial, PersistenceLandscape, HeatKernel, Silhouette, BettiCurve, PairwiseDistance, ForgetDimension
from gtda.plotting import plot_point_cloud, plot_heatmap, plot_diagram
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA, FastICA
from gtda.pipeline import Pipeline 

In [2]:
# Choose if you want to look at EEG or EMG data

#data_type = "EEG" # Does not have an effect yet, will be added later when processing anesthesia data
data_type = "EMG"

In [3]:
# choose individuum

subject = "m294"

In [4]:
label_list = [0, 1, 2, 3, 4]

n_folds = 5 # This should be the same as in the file which creates PDs 

In [41]:
# Load persistence diagrams

test_persistence_diagrams = {} # dictionary with labels as keys, persistence diagrams of the respective classes as values
train_persistence_diagrams = {} # dictionary with labels as keys, persistence diagrams of the respective classes as values
validation_persistence_diagrams = {}

for label in label_list:
    train_persistence_diagrams[label] = {}
    validation_persistence_diagrams[label] = {}


for label in label_list:
    # Final Test set
    test_persistence_diagrams[label]  = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Test/'+str(data_type)+'/PD'+str(label)+'.npy', \
    allow_pickle=True).item() # .item() to convert the dtype to dict again

    # Folds
    for fold in range(n_folds):
        # Train Set
        train_persistence_diagrams[label][fold] = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Train/'+str(data_type)+'/Fold_'+str(fold)+'/Train/PD_Label_'+str(label)+'.npy', \
                allow_pickle=True)
        
        # Validation Set
        validation_persistence_diagrams[label][fold] = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Train/'+str(data_type)+'/Fold_'+str(fold)+'/Validation/PD_Label_'+str(label)+'.npy', \
                allow_pickle=True)

# Summary Statistics

In [94]:
def compute_summary_statistics(persistence_diagrams):
    """
    Compute summary statistics of list of persistence diagrams

    Parameters:
    - persistence_diagrams (list): persistence diagrams

    Returns:
    Tuple of four lists:
    - Persistence Entropy
    - Persistence
    - Betti Numbers
    - Complex Polynomials
    """
    
    PE = PersistenceEntropy()
    AM = Amplitude()
    NP = NumberOfPoints()
    CP = ComplexPolynomial(n_coefficients=1)

    persistence_entropies = []
    amplitudes = []
    nos_points = []
    complex_polynomials = []

    for diagram in persistence_diagrams:
        persistence_entropies.append(PE.fit_transform([diagram]))
        amplitudes.append(AM.fit_transform([diagram]))
        nos_points.append(NP.fit_transform([diagram]))
        #complex_polynomials.append(CP.fit_transform([diagram]))

    return persistence_entropies, amplitudes, nos_points, #complex_polynomials

In [51]:
# Initialize dict for final test set with labels as key
test_feautures = {}

# Initialize dicts for the different folds set with labels as key
train_feautures = {}
validation_feautures = {}


for label in label_list:

    # Initialize a dict for each label for the different folds
    train_feautures[label] = {}
    validation_feautures[label] = {}

    test_feautures[label] = compute_summary_statistics(test_persistence_diagrams[label][0])
    
    for fold in range(n_folds):
        train_feautures[label][fold] = compute_summary_statistics(train_persistence_diagrams[label][fold])
        validation_feautures[label][fold] = compute_summary_statistics(validation_persistence_diagrams[label][fold])

In [97]:
def compute_largest_persistence(persistence_diagrams):
    """
    Computes persistence of the most prominent points of each dimension in each diagram

    Parameters:
    - persistence_diagrams (list): persistence diagrams

    Returns:
    List of 3 lists:
    - List of the largest persistences of homology dimension 0
    - List of the largest persistences of homology dimension 1
    - List of the largest persistences of homology dimension 2 
    """

    largest_persistences = [] # will contain 3 lists for the 3 homology dimensions
    for homology_dimension in [0, 1, 2]:
        largest_persistences_of_hom_dim = []
        for diagram in persistence_diagrams:
            # only look at holes of our homology dimension
            condition = diagram[:, 2] == homology_dimension
            filtered_diagram = diagram[condition]

            if len(filtered_diagram) > 0:
                differences = filtered_diagram[:, 1] - filtered_diagram[:, 0]
                largest_persistences_of_hom_dim.append(np.max(differences))

        largest_persistences.append(largest_persistences_of_hom_dim)

    return largest_persistences

In [98]:
# Initialize dict for final test set with labels as key
test_largest_persistences = {}

# Initialize dicts for the different folds set with labels as key
train_largest_persistences = {}
validation_largest_persistences = {}


for label in label_list:

    test_largest_persistences[label] = compute_largest_persistence(test_persistence_diagrams[label][0])

    # Initialize a dict for each label for the different folds
    train_largest_persistences[label] = {}
    validation_largest_persistences[label] = {}

    
    for fold in range(n_folds):
        train_largest_persistences[label][fold] = compute_largest_persistence(train_persistence_diagrams[label][fold])
        validation_largest_persistences[label][fold] = compute_largest_persistence(validation_persistence_diagrams[label][fold])

# Concatenate Features to one DataFrame

In [99]:
def choose_column_in_matrix(matrix, i):
    return [row[0][i] for row in matrix]

In [100]:
def create_feature_df(subject, data_type, persistence_entropies, amplitudes, nos_points, persistences, label, train):
    """
    Create DataFrame for each label from features

    Parameters:
    - persistence_entropies (list): persistence entropies
    - amplitudes (list): amplitudes
    - nos_points (list): number of points
    - label (int): Label for which we want to create a dataframe. 1, 3, 5 or 7.

    Returns:
    - Feature DataFrame (DataFrame)
    """
    
    feature_df = pd.DataFrame()

    # All 3 columns (corresponding to hole dimensions)
    feature_df[str(data_type)+"_Persistence Entropy_Dim_0"] = choose_column_in_matrix(list(persistence_entropies), 0)
    feature_df[str(data_type)+"_Persistence Entropy_Dim_1"] = choose_column_in_matrix(list(persistence_entropies), 1)
    feature_df[str(data_type)+"_Persistence Entropy_Dim_2"] = choose_column_in_matrix(list(persistence_entropies), 2)

    # All 3 columns (corresponding to hole dimensions)
    feature_df[str(data_type)+"_Amplitude_Dim_0"] = choose_column_in_matrix(list(amplitudes), 0)
    feature_df[str(data_type)+"_Amplitude_Dim_1"] = choose_column_in_matrix(list(amplitudes), 1)
    feature_df[str(data_type)+"_Amplitude_Dim_2"] = choose_column_in_matrix(list(amplitudes), 2)

    # All 3 columns (corresponding to hole dimensions)
    feature_df[str(data_type)+"_No_Points_Dim_0"] = choose_column_in_matrix(list(nos_points), 0)
    feature_df[str(data_type)+"_No_Points_Dim_1"] = choose_column_in_matrix(list(nos_points), 1)
    feature_df[str(data_type)+"_No_Points_Dim_2"] = choose_column_in_matrix(list(nos_points), 2)


    feature_df[str(data_type)+"_Largest_Persistence_Dim_0"] = persistences[0]
    feature_df[str(data_type)+"_Largest_Persistence_Dim_1"] = persistences[1]
    feature_df[str(data_type)+"_Largest_Persistence_Dim_2"] = persistences[2]

    # Label
    feature_df["Label"] = label

    # Subject
    feature_df["Subject"] = subject

    feature_df["Train"] = train


    return feature_df

In [103]:
test_dataframes= {}

train_dataframes = {}
validation_dataframes = {}



for label in label_list:
    # Final test set
    persistence_entropies = test_feautures[label][0]
    amplitudes = test_feautures[label][1]
    nos_points = test_feautures[label][2]

    test_dataframes[label] = create_feature_df(subject, data_type, persistence_entropies, amplitudes, nos_points, \
                                                test_largest_persistences[label], label, False)

    # Initialize dict with folds as key for each label
    train_dataframes[label] = {}
    validation_dataframes[label] = {}

    # Folds
    for fold in range(n_folds):
        # Train sets
        persistence_entropies = train_feautures[label][fold][0]
        amplitudes = train_feautures[label][fold][1]
        nos_points = train_feautures[label][fold][2]
        
        train_dataframes[label][fold] = create_feature_df(subject, data_type, persistence_entropies, amplitudes, nos_points, \
                                                train_largest_persistences[label][fold], label, False)


        # Validation sets
        persistence_entropies = validation_feautures[label][fold][0]
        amplitudes = validation_feautures[label][fold][1]
        nos_points = validation_feautures[label][fold][2]
        
        validation_dataframes[label][fold] = create_feature_df(subject, data_type, persistence_entropies, amplitudes, nos_points, \
                                                validation_largest_persistences[label][fold], label, False)






In [105]:
# Concatenate and save features of final test set
test_feature_df = pd.concat([test_dataframes[0], test_dataframes[1], test_dataframes[2], test_dataframes[3], test_dataframes[4]], ignore_index=True)
test_feature_df.to_csv("Features/"+str(subject)+"/Test/"+str(data_type)+"/Topological_Summary_Statistics.csv")


# Concatenate and save features of for single folds
for fold in range(n_folds):
    # Train
    train_feature_df = pd.concat([test_dataframes[0], test_dataframes[1], test_dataframes[2], test_dataframes[3], test_dataframes[4]], ignore_index=True)
    train_feature_df.to_csv("Features/"+str(subject)+"/Train/"+str(data_type)+"/Fold_"+str(fold)+"/Train/Topological_Summary_Statistics.csv")

    # Validation
    validation_feature_df = pd.concat([test_dataframes[0], test_dataframes[1], test_dataframes[2], test_dataframes[3], test_dataframes[4]], ignore_index=True)
    validation_feature_df.to_csv("Features/"+str(subject)+"/Train/"+str(data_type)+"/Fold_"+str(fold)+"/Validation/Topological_Summary_Statistics.csv")
